In [33]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd

In [50]:
sc = SparkContext.getOrCreate()
spark = pyspark.sql.SparkSession.builder.appName("DeltaTest") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.7.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

from delta.tables import *

In [6]:
##Batch
data = spark.range(0, 5)
data.write.format("delta").save(r"C:\Users\kndoa\OneDrive\Desktop\Data_Science\CovidDeltaLake\test")

In [7]:
df = spark.read.format("delta").load(r"C:\Users\kndoa\OneDrive\Desktop\Data_Science\CovidDeltaLake\test")
df.show()

+---+
| id|
+---+
|  0|
|  3|
|  1|
|  2|
|  4|
+---+



In [8]:
##Stream
streamingDf = spark.readStream.format("rate").load()
stream = streamingDf.selectExpr("value as id").writeStream.format("delta").option("checkpointLocation", "/tmp/checkpoint").start("/tmp/delta-table")

In [9]:
stream2 = spark.readStream.format("delta").load("/tmp/delta-table").writeStream.format("console").start()

In [2]:
import boto3

In [3]:
s3 = boto3.resource('s3')

In [4]:
for bucket in s3.buckets.all():
    print(bucket.name)

aws-emr-resources-192677340977-us-east-2
aws-glue-scripts-192677340977-us-east-2
aws-glue-temporary-192677340977-us-east-2
aws-logs-192677340977-us-east-2
aws-logs-192677340977-us-west-2
covid-19-lake
covid-delta-lake
knd-udacity-data-lake-project
nifi-bucket-doan
pxml-objects
testparquetdata


In [13]:
bucketname = ""
s3 = boto3.resource('s3')
#obj = s3.Object("covid19-lake", "rearc-covid-19-testing-data/csv/states_daily/states_daily.csv")
obj = s3.Object("covid19-lake", "static-datasets/json/state-abv/part-00000-22cb15f0-19dd-49d4-981b-e628263fbb82-c000.json")
body = obj.get()['Body'].read()

In [15]:
type(body)

bytes

In [24]:
states_json_string = bytes.decode(body)

In [40]:
states_json_string

'{"State":"Alabama","Abbreviation":"AL"}\n{"State":"Alaska","Abbreviation":"AK"}\n{"State":"Arizona","Abbreviation":"AZ"}\n{"State":"Arkansas","Abbreviation":"AR"}\n{"State":"California","Abbreviation":"CA"}\n{"State":"Colorado","Abbreviation":"CO"}\n{"State":"Connecticut","Abbreviation":"CT"}\n{"State":"Delaware","Abbreviation":"DE"}\n{"State":"District of Columbia","Abbreviation":"DC"}\n{"State":"Florida","Abbreviation":"FL"}\n{"State":"Georgia","Abbreviation":"GA"}\n{"State":"Hawaii","Abbreviation":"HI"}\n{"State":"Idaho","Abbreviation":"ID"}\n{"State":"Illinois","Abbreviation":"IL"}\n{"State":"Indiana","Abbreviation":"IN"}\n{"State":"Iowa","Abbreviation":"IA"}\n{"State":"Kansas","Abbreviation":"KS"}\n{"State":"Kentucky","Abbreviation":"KY"}\n{"State":"Louisiana","Abbreviation":"LA"}\n{"State":"Maine","Abbreviation":"ME"}\n{"State":"Montana","Abbreviation":"MT"}\n{"State":"Nebraska","Abbreviation":"NE"}\n{"State":"Nevada","Abbreviation":"NV"}\n{"State":"New Hampshire","Abbreviation"

In [28]:
df = pd.read_json(states_json_string, lines = True)

In [51]:
import json
json_str_dump = json.dumps(states_json_string)

jsonRDD = sc.parallelize([json_str_dump])

In [66]:
jsonRDD.take(10)[0]

'"{\\"State\\":\\"Alabama\\",\\"Abbreviation\\":\\"AL\\"}\\n{\\"State\\":\\"Alaska\\",\\"Abbreviation\\":\\"AK\\"}\\n{\\"State\\":\\"Arizona\\",\\"Abbreviation\\":\\"AZ\\"}\\n{\\"State\\":\\"Arkansas\\",\\"Abbreviation\\":\\"AR\\"}\\n{\\"State\\":\\"California\\",\\"Abbreviation\\":\\"CA\\"}\\n{\\"State\\":\\"Colorado\\",\\"Abbreviation\\":\\"CO\\"}\\n{\\"State\\":\\"Connecticut\\",\\"Abbreviation\\":\\"CT\\"}\\n{\\"State\\":\\"Delaware\\",\\"Abbreviation\\":\\"DE\\"}\\n{\\"State\\":\\"District of Columbia\\",\\"Abbreviation\\":\\"DC\\"}\\n{\\"State\\":\\"Florida\\",\\"Abbreviation\\":\\"FL\\"}\\n{\\"State\\":\\"Georgia\\",\\"Abbreviation\\":\\"GA\\"}\\n{\\"State\\":\\"Hawaii\\",\\"Abbreviation\\":\\"HI\\"}\\n{\\"State\\":\\"Idaho\\",\\"Abbreviation\\":\\"ID\\"}\\n{\\"State\\":\\"Illinois\\",\\"Abbreviation\\":\\"IL\\"}\\n{\\"State\\":\\"Indiana\\",\\"Abbreviation\\":\\"IN\\"}\\n{\\"State\\":\\"Iowa\\",\\"Abbreviation\\":\\"IA\\"}\\n{\\"State\\":\\"Kansas\\",\\"Abbreviation\\":\\"KS\\"

In [69]:
states = spark.createDataFrame(df)

In [70]:
states.show(10)

+--------------------+------------+
|               State|Abbreviation|
+--------------------+------------+
|             Alabama|          AL|
|              Alaska|          AK|
|             Arizona|          AZ|
|            Arkansas|          AR|
|          California|          CA|
|            Colorado|          CO|
|         Connecticut|          CT|
|            Delaware|          DE|
|District of Columbia|          DC|
|             Florida|          FL|
+--------------------+------------+
only showing top 10 rows



In [80]:
s3 = boto3.resource('s3')
#obj = s3.Object("covid19-lake", "rearc-covid-19-testing-data/csv/states_daily/states_daily.csv")
obj = s3.Object("covid19-lake", "rearc-covid-19-nyt-data-in-usa/json/us-counties/part-00000-19b49410-a7b8-41bb-8e84-d187ff6d2477-c000.json")
body = obj.get()['Body'].read()

In [111]:
s3 = boto3.client("s3")
all_objects = s3.list_objects(Bucket = 'covid19-lake').filter('/rearc-covid-19-nyt-data-in-usa/json/us-counties') 

AttributeError: 'dict' object has no attribute 'filter'

In [121]:
s3 = boto3.resource('s3')
my_bucket = s3.Bucket('covid19-lake')
for obj in my_bucket.objects.filter(Prefix='/rearc-covid-19-nyt-data-in-usa/json/us-counties/'):
    print(obj)

In [122]:
all_objects

8

In [123]:
contents = all_objects['Contents']

In [126]:
all_objects

{'ResponseMetadata': {'RequestId': '629A2DB577736D24',
  'HostId': 'RM16O4Tv0u6MPc5ceoAQTJBLoxPmtONEFIktG38nqN+26k0L/37ZRWtNOjfp4kpP2b8rcy7kXCQ=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'RM16O4Tv0u6MPc5ceoAQTJBLoxPmtONEFIktG38nqN+26k0L/37ZRWtNOjfp4kpP2b8rcy7kXCQ=',
   'x-amz-request-id': '629A2DB577736D24',
   'date': 'Wed, 16 Sep 2020 00:16:27 GMT',
   'x-amz-bucket-region': 'us-east-2',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'IsTruncated': True,
 'Marker': '',
 'Contents': [{'Key': 'alleninstitute/CORD19/comprehendmedical/',
   'LastModified': datetime.datetime(2020, 3, 24, 19, 56, 58, tzinfo=tzutc()),
   'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'Size': 0,
   'StorageClass': 'STANDARD'},
  {'Key': 'alleninstitute/CORD19/comprehendmedical/comprehend_medical.json',
   'LastModified': datetime.datetime(2020, 3, 31, 1, 23, 34, tzinfo=tzutc()),
   'ETag': '"9c0f88001f602974bbb9cf4

In [124]:
len(contents)

1000

In [136]:
def get_matching_s3_keys(bucket, prefix='', suffix=''):
    """
    Generate the keys in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch keys that start with this prefix (optional).
    :param suffix: Only fetch keys that end with this suffix (optional).
    """
    s3 = boto3.client('s3')
    kwargs = {'Bucket': bucket}

    # If the prefix is a single string (not a tuple of strings), we can
    # do the filtering directly in the S3 API.
    if isinstance(prefix, str):
        kwargs['Prefix'] = prefix

    while True:

        # The S3 API response is a large blob of metadata.
        # 'Contents' contains information about the listed objects.
        resp = s3.list_objects_v2(**kwargs)
        for obj in resp['Contents']:
            key = obj['Key']
            if key.startswith(prefix) and key.endswith(suffix):
                yield key

        # The S3 API is paginated, returning up to 1000 keys at a time.
        # Pass the continuation token into the next response, until we
        # reach the final page (when this field is missing).
        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break

In [148]:
get_matching_s3_keys('covid19-lake', prefix='/rearc-covid-19-nyt-data-in-usa/json/us-counties/', suffix = 'json')

<generator object get_matching_s3_keys at 0x000002079EC3AE48>

In [159]:
for key in get_matching_s3_keys('covid19-lake', prefix='/rearc-covid-19-nyt-data-in-usa/json/us-counties/', suffix = 'json'):
    print(key)

KeyError: 'Contents'

In [155]:
prefix = "/rearc-covid-19-nyt-data-in-usa/json/us-counties/"
suffix = ".json"
resp = s3.list_objects_v2(Bucket = 'covid19-lake')
for obj in resp['Contents']:
    key = obj['Key']
    print(obj)

{'Key': 'alleninstitute/CORD19/comprehendmedical/', 'LastModified': datetime.datetime(2020, 3, 24, 19, 56, 58, tzinfo=tzutc()), 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', 'Size': 0, 'StorageClass': 'STANDARD'}
{'Key': 'alleninstitute/CORD19/comprehendmedical/comprehend_medical.json', 'LastModified': datetime.datetime(2020, 3, 31, 1, 23, 34, tzinfo=tzutc()), 'ETag': '"9c0f88001f602974bbb9cf44b9b1b5bc"', 'Size': 14136397, 'StorageClass': 'STANDARD'}
{'Key': 'alleninstitute/CORD19/json/metadata/part-00000-a51649f8-8d21-4dd9-a339-0a3e6a0fe3de-c000.json', 'LastModified': datetime.datetime(2020, 9, 16, 0, 33, 8, tzinfo=tzutc()), 'ETag': '"e6a05e805c8bba2a412366c796f5a953"', 'Size': 78485736, 'StorageClass': 'STANDARD'}
{'Key': 'alleninstitute/CORD19/json/metadata/part-00001-a51649f8-8d21-4dd9-a339-0a3e6a0fe3de-c000.json', 'LastModified': datetime.datetime(2020, 9, 16, 0, 33, 8, tzinfo=tzutc()), 'ETag': '"6ab439f929cc2be25e76fd2d12a83161"', 'Size': 78427247, 'StorageClass': 'STANDARD'}
{'K